I want to be able to launch a bunch of experiments when I am away and plot their results. I think I need:
- A callback class that stores metrics each epoch
- A way to store models with all the parameters
- A way to store the optimizer's params
- A way to store the fitter parameters
- A way to reproduce an experiment easily
- Include time() in the metrics

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from fae import dataset_loader
from fae.training_utils import NBatchLogger

import numpy as np
import tensorflow as tf

import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam, SGD
from keras import backend as K

Using TensorFlow backend.


In [3]:
from keras.callbacks import Callback
import time

class ExperimentLogger(Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self):
        self.exp_log = list()
        self.start_ts = time.time()

    def on_epoch_end(self, epoch, logs={}):
        logs['timestamp'] = time.time()-self.start_ts
        self.exp_log.append(logs)


In [4]:
dropout=0.1
core_size=128

model = Sequential()
model.add(Dense(core_size, activation='sigmoid', input_shape=(14,)))
model.add(Dropout(dropout))
model.add(Dense(core_size, activation='sigmoid'))
model.add(Dropout(dropout))
model.add(Dense(core_size, activation='sigmoid'))
model.add(Dropout(dropout))
model.add(Dense(core_size, activation='sigmoid'))
model.add(Dropout(dropout))
model.add(Dense(4, activation='linear'))

model.summary()

W1023 00:10:12.158381 139623787867136 deprecation_wrapper.py:119] From /home/yves/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1023 00:10:12.171562 139623787867136 deprecation_wrapper.py:119] From /home/yves/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1023 00:10:12.174032 139623787867136 deprecation_wrapper.py:119] From /home/yves/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1023 00:10:12.193179 139623787867136 deprecation_wrapper.py:119] From /home/yves/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default inst

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1920      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
__________

In [5]:
config = model.get_config()

In [6]:
Sequential.from_config(config).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1920      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
__________

In [7]:
opt = Adam(lr=0.004, amsgrad=True)
model.compile(loss='mean_squared_error',
  optimizer=opt,
  metrics=['accuracy'])


W1023 00:10:12.469532 139623787867136 deprecation_wrapper.py:119] From /home/yves/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [8]:
opt.get_config()

W1023 00:10:12.486529 139623787867136 deprecation_wrapper.py:119] From /home/yves/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



{'lr': 0.004000000189989805,
 'beta_1': 0.8999999761581421,
 'beta_2': 0.9990000128746033,
 'decay': 0.0,
 'epsilon': 1e-07,
 'amsgrad': True}

In [9]:
Adam.from_config(opt.get_config())

In [22]:
pp_input, pp_output, mp_input, mp_output = dataset_loader.load_dataset("../datasets/real/2019-10-18/raw/")

In [23]:
el = ExperimentLogger()
model.fit(x=mp_input, y=mp_output,
    batch_size=10000,
    epochs=50,
    verbose=0,
    shuffle=True,
    validation_split=0.1,
    callbacks=[NBatchLogger(display=10), el])

Epoch: 10/50 ...  - loss: 0.0333 - acc: 0.3628 - val_loss: 0.0191 - val_acc: 0.5296
Epoch: 20/50 ...  - loss: 0.0294 - acc: 0.4292 - val_loss: 0.0205 - val_acc: 0.5296
Epoch: 30/50 ...  - loss: 0.0267 - acc: 0.3953 - val_loss: 0.0192 - val_acc: 0.5296
Epoch: 40/50 ...  - loss: 0.0246 - acc: 0.4330 - val_loss: 0.0198 - val_acc: 0.5296


In [24]:
list(enumerate(el.exp_log))

[(0,
  {'val_loss': 0.022714687511324883,
   'val_acc': 0.5296208262443542,
   'loss': 0.03724024444818497,
   'acc': 0.4200817048549652,
   'timestamp': 0.05315208435058594}),
 (1,
  {'val_loss': 0.02310163900256157,
   'val_acc': 0.5296208262443542,
   'loss': 0.03703784570097923,
   'acc': 0.4261431097984314,
   'timestamp': 0.09818053245544434}),
 (2,
  {'val_loss': 0.022839054465293884,
   'val_acc': 0.5296208262443542,
   'loss': 0.03678693249821663,
   'acc': 0.4283831715583801,
   'timestamp': 0.14828133583068848}),
 (3,
  {'val_loss': 0.02208639122545719,
   'val_acc': 0.5296208262443542,
   'loss': 0.03601698949933052,
   'acc': 0.420872300863266,
   'timestamp': 0.19316506385803223}),
 (4,
  {'val_loss': 0.021031750366091728,
   'val_acc': 0.5296208262443542,
   'loss': 0.035706836730241776,
   'acc': 0.4181051552295685,
   'timestamp': 0.23741388320922852}),
 (5,
  {'val_loss': 0.02008625492453575,
   'val_acc': 0.5296208262443542,
   'loss': 0.034055259078741074,
   'acc':

In [25]:
import pickle, zlib

In [26]:
len(pickle.dumps(el.exp_log))

8496

In [27]:
len(zlib.compress(pickle.dumps(el.exp_log)))

3134

In [28]:
r = zlib.compress(pickle.dumps(el.exp_log))

In [29]:
len(repr(pickle.loads(zlib.decompress(r))))

7813

Ok I think I found out what I wanted. Now to write a class `Experiment`.

Done. Let's test it.

In [30]:
from fae.training_utils import Experiment, NBatchLogger
from fae import dataset_loader

In [31]:
import random
import numpy as np
import tensorflow as tf

def reset_seed(seed_value=0):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.set_random_seed(seed_value)

In [32]:
model.reset_states()
reset_seed()
e=Experiment(model=model, optimizer=opt, 
             fit_params = { 'batch_size':10000,
                            'epochs':500,
                            'verbose':0,
                            'shuffle':True,
                            'validation_split':0.1},
            compile_params={'loss'    :'mean_squared_error',
                            'metrics' :['mean_squared_error']},
            data_loader = 'fae.dataset_loader.load_dataset("../datasets/real/2019-10-18/raw/")[2:4]')


In [33]:
reset_seed()
e.run(display=10)

Epoch: 10/500 ...  - loss: 0.1326 - mean_squared_error: 0.1326 - val_loss: 0.1585 - val_mean_squared_error: 0.1585
Epoch: 20/500 ...  - loss: 0.0822 - mean_squared_error: 0.0822 - val_loss: 0.0456 - val_mean_squared_error: 0.0456
Epoch: 30/500 ...  - loss: 0.0574 - mean_squared_error: 0.0574 - val_loss: 0.0422 - val_mean_squared_error: 0.0422
Epoch: 40/500 ...  - loss: 0.0458 - mean_squared_error: 0.0458 - val_loss: 0.0198 - val_mean_squared_error: 0.0198
Epoch: 50/500 ...  - loss: 0.0398 - mean_squared_error: 0.0398 - val_loss: 0.0249 - val_mean_squared_error: 0.0249
Epoch: 60/500 ...  - loss: 0.0366 - mean_squared_error: 0.0366 - val_loss: 0.0186 - val_mean_squared_error: 0.0186
Epoch: 70/500 ...  - loss: 0.0332 - mean_squared_error: 0.0332 - val_loss: 0.0206 - val_mean_squared_error: 0.0206
Epoch: 80/500 ...  - loss: 0.0301 - mean_squared_error: 0.0301 - val_loss: 0.0190 - val_mean_squared_error: 0.0190
Epoch: 90/500 ...  - loss: 0.0277 - mean_squared_error: 0.0277 - val_loss: 0.019

In [34]:
e.save_name

'./experiments//exp_20191023_00-10-49.pickle'

In [35]:
reset_seed()
e2=Experiment.from_file(e.save_name)
reset_seed()
e2.run(display=10)

Epoch: 10/500 ...  - loss: 0.1562 - mean_squared_error: 0.1562 - val_loss: 0.1261 - val_mean_squared_error: 0.1261
Epoch: 20/500 ...  - loss: 0.0775 - mean_squared_error: 0.0775 - val_loss: 0.0259 - val_mean_squared_error: 0.0259
Epoch: 30/500 ...  - loss: 0.0515 - mean_squared_error: 0.0515 - val_loss: 0.0268 - val_mean_squared_error: 0.0268
Epoch: 40/500 ...  - loss: 0.0401 - mean_squared_error: 0.0401 - val_loss: 0.0185 - val_mean_squared_error: 0.0185
Epoch: 50/500 ...  - loss: 0.0343 - mean_squared_error: 0.0343 - val_loss: 0.0194 - val_mean_squared_error: 0.0194
Epoch: 60/500 ...  - loss: 0.0299 - mean_squared_error: 0.0299 - val_loss: 0.0207 - val_mean_squared_error: 0.0207
Epoch: 70/500 ...  - loss: 0.0271 - mean_squared_error: 0.0271 - val_loss: 0.0190 - val_mean_squared_error: 0.0190
Epoch: 80/500 ...  - loss: 0.0250 - mean_squared_error: 0.0250 - val_loss: 0.0202 - val_mean_squared_error: 0.0202
Epoch: 90/500 ...  - loss: 0.0226 - mean_squared_error: 0.0226 - val_loss: 0.019

In [36]:
reset_seed()
e3=Experiment.from_file(e.save_name)
reset_seed()
e3.run(display=10)

Epoch: 10/500 ...  - loss: 0.1419 - mean_squared_error: 0.1419 - val_loss: 0.0944 - val_mean_squared_error: 0.0944
Epoch: 20/500 ...  - loss: 0.0636 - mean_squared_error: 0.0636 - val_loss: 0.0181 - val_mean_squared_error: 0.0181
Epoch: 30/500 ...  - loss: 0.0457 - mean_squared_error: 0.0457 - val_loss: 0.0190 - val_mean_squared_error: 0.0190
Epoch: 40/500 ...  - loss: 0.0390 - mean_squared_error: 0.0390 - val_loss: 0.0216 - val_mean_squared_error: 0.0216
Epoch: 50/500 ...  - loss: 0.0344 - mean_squared_error: 0.0344 - val_loss: 0.0180 - val_mean_squared_error: 0.0180
Epoch: 60/500 ...  - loss: 0.0298 - mean_squared_error: 0.0298 - val_loss: 0.0212 - val_mean_squared_error: 0.0212
Epoch: 70/500 ...  - loss: 0.0265 - mean_squared_error: 0.0265 - val_loss: 0.0194 - val_mean_squared_error: 0.0194
Epoch: 80/500 ...  - loss: 0.0244 - mean_squared_error: 0.0244 - val_loss: 0.0194 - val_mean_squared_error: 0.0194
Epoch: 90/500 ...  - loss: 0.0220 - mean_squared_error: 0.0220 - val_loss: 0.020

In [ ]:
e.el.exp_log[-2:]